In [18]:
print('python is easy!')

python is easy!


In [19]:
## Computing planetary positions based on Keplerian elements. Done in two steps:
## 1) Compute rectangular ecliptic coordinates of planet in heliocentric system
## 2) Compute rectangular ecliptic coordinates of earth in heliocentric system
## 3) Add 1 and 2 to get geocentric rectangular ecliptic coordinates

## Standish document https://ssd.jpl.nasa.gov/txt/aprx_pos_planets.pdf provides widely used
## Keplerian elements. However it doesnt have elements for earth itself needed in Step 2. The 
## closest that can be used is one for Earth-Moon barymetric.
## Tutorial by Paul Schlyter lists another set (probably again from NASA - didnt check) that 
## includes the elements for earth. This is the second way implemented in this notebook

## Output is compared to Stellarium 

In [20]:
import math
import numpy as np
import pandas as pd
import datetime
from astropy.time import Time
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
def cos(deg):
    return math.cos(deg*math.pi/180)

def sin(deg):
    return math.sin(deg*math.pi/180)

## borrowed from Paul Schlyter's tutorial - regular math.atan doesnt have these properties
def arctan_stleimlen(x,y):
    if x==0:
        if y==0:
            return 0
        elif y>0:
            return 90
        else:
            return -90
    elif x>0:
        return math.atan(y/x)*180/math.pi
    else: ## x<0
        if y>=0:
            return math.atan(y/x)*180/math.pi + 180
        else:
            return math.atan(y/x)*180/math.pi - 180

def get_T(julian):
    return (julian-2451545)/36525

def get_d(julian):
    return julian-2451543.5

def solve_E(M, e):
    TOLERANCE = 1E-6
    e_deg = e*180/math.pi
    
    ## E_ is in degree
    E_ = M + e_deg*sin(M)
    
    for n in range(1,500):
        delta_M = M - (E_ - e_deg*sin(E_))
        delta_E = delta_M/(1 - e*cos(E_))
        if (abs(delta_E) < TOLERANCE):
            ## print(f'converged after {n} iterations')
            break
        E_ += delta_E
        
        
    return E_

In [22]:
def do_Standish(jd,a=1.52371243,e=0.09336511,I=1.85181869,L=-4.56813164,wbar=-23.91744784,N=49.71320984,
                acy=0.00000097,ecy=0.00009149,Icy=-0.00724757,Lcy=19140.29934243,wbarcy=0.45223625,Ncy=-0.26852431):
    
    T = get_T(jd)

    a += acy*T
    e += ecy*T
    I += Icy*T
    L += Lcy*T
    wbar += wbarcy*T
    N += Ncy*T

    ## print(f'a {a}, e {e}, I {I}, L {L}, wbar {wbar}, N {N}')
    
    ## wbar = w+N
    w = wbar - N
    M = L - wbar
    if (M<0):
        M = -(-M % 180)
    else:
        M = M % 180

    ## print(f'w {w}, M {M}')

    ## need to find E such that M = E - 57.29578e sinE
    E = solve_E(M,e)
    ## print(f'E {E}')
    
    ## X-axis aligned to perihilion!!
    x_prime = a*(cos(E)-e)
    y_prime = a*math.sqrt(1-e*e)*sin(E)
    ## print(f'x_prime {x_prime}, y_prime {y_prime}')
    
    
    ## X-axis aligned to equinox!!
    x_ecl = x_prime*(cos(w)*cos(N)-sin(w)*sin(N)*cos(I)) - y_prime*(sin(w)*cos(N)+cos(w)*sin(N)*cos(I))
    y_ecl = x_prime*(cos(w)*sin(N)+sin(w)*cos(N)*cos(I)) - y_prime*(sin(w)*sin(N)-cos(w)*cos(N)*cos(I))
    z_ecl = x_prime*sin(w)*sin(I) + y_prime*cos(w)*sin(I)

    return x_ecl,y_ecl,z_ecl

In [23]:
## julian date for 19-4-1990 using Stellarium
jd = 2448000.4167

## julian for 1-1-2004 using Stellarium
jd = 2453005.458333

## julian for 3000 BC
## jd = 623846.480405

## MARS
a=1.52371243
e=0.09336511
I=1.85181869
L=-4.56813164
wbar=-23.91744784
N=49.71320984
acy=0.00000097
ecy=0.00009149
Icy=-0.00724757
Lcy=19140.29934243
wbarcy=0.45223625
Ncy=-0.26852431

x_mars_ecl, y_mars_ecl, z_mars_ecl, = do_Standish(jd,a,e,I,L,wbar,N,acy,ecy,Icy,Lcy,wbarcy,Ncy)
print(f'Standish MARS for 19/4/1990: x_mars_ecl {x_mars_ecl}, y_mars_ecl {y_mars_ecl}, z_mars_ecl {z_mars_ecl}')

Standish MARS for 19/4/1990: x_mars_ecl 0.9301400688790877, y_mars_ecl 1.1438512668497707, z_mars_ecl 0.0009820579535514719


In [24]:
## EARTH-MOON B as proy for SUN as per Standish 
a=1.00000018
e=0.01673163
I=-0.00054346
L=100.46691572
wbar=102.93005885
N=-5.11260389
acy=-0.00000003
ecy=-0.00003661
Icy=-0.01337178
Lcy=35999.37306329
wbarcy=0.31795260
Ncy=-0.24123856

x_earth_ecl, y_earth_ecl, z_earth_ecl = do_Standish(jd,a,e,I,L,wbar,N,acy,ecy,Icy,Lcy,wbarcy,Ncy)
print(f'Standish SUN/EARTH for 19/4/1990: x_earth_ecl {x_earth_ecl}, y_earth_ecl {y_earth_ecl}, z_earth_ecl {z_earth_ecl}')


Standish SUN/EARTH for 19/4/1990: x_earth_ecl 0.17667851047751298, y_earth_ecl -1.0012388073980374, z_earth_ecl 1.8468220392266638e-05


In [25]:
## from Earth it is sum of sun and planet
x_standish_ecl = x_mars_ecl + x_earth_ecl
y_standish_ecl = y_mars_ecl + y_earth_ecl
z_standish_ecl = z_mars_ecl + z_earth_ecl

## now rotate by obliquity of Eclipise ie. 23 degree tilt to get equatorial
oblecl = 23.4393
x_standish = x_standish_ecl
y_standish = y_standish_ecl * cos(oblecl) - z_standish_ecl * sin(oblecl)
z_standish = y_standish_ecl * sin(oblecl) + z_standish_ecl * cos(oblecl)

RA_standish = arctan_stleimlen(x_standish,y_standish)
Decl_standish = arctan_stleimlen(math.sqrt(x_standish*x_standish + y_standish*y_standish),z_standish)

if (RA_standish<0):
    RA_standish += 360
RA_standish_hours = RA_standish/15
print(f'x {x_standish}, y {y_standish}, z {z_standish}')
print(f'RA in hours {RA_standish_hours}, Decl {Decl_standish}')

x 1.1068185793566008, y 0.13044637797541206, z 0.057645963575890996
RA in hours 0.4481136096346479, Decl 2.960961452655975


In [26]:
RA_standish

6.721704144519719

In [27]:
def rev(x):
    rev = x - math.trunc(x/360.0)*360.0
    if rev<0.0:
        rev=rev+360.0
      
    return rev

In [50]:
def do_stjarnhimlen(jd,a,e,w,M,a_d,e_d,w_d,M_d,N,N_d,I,I_d):
    d = get_d(jd)
    w = w + w_d*d
    a = a + a_d*d
    e = e + e_d*d
    M = M + M_d*d
    N = N + N_d*d
    I = I + I_d*d
    
    ## take modulo 360
    M = rev(M)
    print(f'For date N I w a e M {N}, {I}, {w}, {a}, {e}, {M}')
    
    L = (w+M+N)%360
    
    E = solve_E(M,e)
    ## print(f'E numerical {E}')
    
    ## X-axis aligned to perihilion!!
    x = a * (cos(E)-e)
    y = a * (sin(E)*math.sqrt(1-e*e))
    ## print(f'x {x}, y {y}')
    
    v_nu = arctan_stleimlen(x,y)
    r = math.sqrt(x*x + y*y)
    ## print(f'v_nu {v_nu}, r {r}')
    lon = (v_nu+w)%360
    
    ## For Sun N=0,i=0, it will reduce to
    ## x_helio_ecl = r * cos(lon)
    ## y_helio_ecl = r * sin(lon)
    ## z_helio_ecl = 0.0
    
    ## ecliptic rectangular, also called xh,yh,zh
    xeclip = r * ( cos(N) * cos(lon) - sin(N) * sin(lon) * cos(I) )
    yeclip = r * ( sin(N) * cos(lon) + cos(N) * sin(lon) * cos(I) )
    zeclip = r * sin(lon) * sin(I)
    
    ## here X is pointing to first point of Aries
    
    return xeclip, yeclip, zeclip

In [51]:
## STJARNHIMLEN method
## SUN
w = 282.9494
w_d=4.70935E-5
a=1
a_d=0
e=0.016709 
e_d=-1.15E-9
M=356.0470
M_d=0.9856002585
N=0
N_d=0
I=0
I_d=0
x_sun_helio_ecl,y_sun_helio_ecl,z_sun_helio_ecl = do_stjarnhimlen(jd,a,e,w,M,a_d,e_d,w_d,M_d,N,N_d,I,I_d)

print(f'Stjarnhimlen for SUN: x_sun_helio_ecl {x_sun_helio_ecl}, y_sun_helio_ecl {y_sun_helio_ecl}, z_sun_helio_ecl {z_sun_helio_ecl}')


For date N I w a e M 0.0, 0.0, 283.01824873475516, 1.0, 0.01670731874791705, 356.9535109211031
Stjarnhimlen for SUN: x_sun_helio_ecl 0.16851716912216655, y_sun_helio_ecl -0.9687695732422757, z_sun_helio_ecl -0.0


In [41]:
## MARS
N=49.5574
N_d=2.11081E-5
I=1.8497
I_d=-1.78E-8
w=286.5016
w_d=2.92961E-5
a=1.523688
a_d=0
e=0.093405
e_d=2.516E-9
M=18.6021
M_d=0.5240207766
x_mars_helio_ecl,y_mars_helio_ecl,z_mars_helio_ecl = do_stjarnhimlen(jd,a,e,w,M,a_d,e_d,w_d,M_d,N,N_d,I,I_d)
print(f'Stjarnhimlen for Mars 19/4/1990: x_mars_helio_ecl {x_mars_helio_ecl}, y_mars_helio_ecl {y_mars_helio_ecl}, z_mars_helio_ecl {z_mars_helio_ecl}')

## from Earth it is sum of x_sun_helio_ecl and x_mars_helio_ecl
x_geo_ecl = x_mars_helio_ecl + x_sun_helio_ecl
y_geo_ecl = y_mars_helio_ecl + y_sun_helio_ecl
z_geo_ecl = z_mars_helio_ecl + z_sun_helio_ecl

## now rotate by obliquity of Eclipise ie. 23 degree tilt to get equatorial
oblecl = 23.4393
x_geo_eq = x_geo_ecl
y_geo_eq = y_geo_ecl * cos(oblecl) - z_geo_ecl * sin(oblecl)
z_geo_eq = y_geo_ecl * sin(oblecl) + z_geo_ecl * cos(oblecl)

RA = arctan_stleimlen(x_geo_eq,y_geo_eq)
Decl = arctan_stleimlen(math.sqrt(x_geo_eq*x_geo_eq + y_geo_eq*y_geo_eq),z_geo_eq)
#print(f'Stjarnhimlen for Mars 19/4/1990: x_geo_eq {x_geo_eq}, y_geo_eq {y_geo_eq}, z_geo_eq {z_geo_eq}, RA {RA}, decl {Decl}')

if (RA<0):
    RA += 360
RA_hours = RA/15

hours = int(RA_hours)
minutes = (RA_hours*60) % 60
seconds = (RA_hours*3600) % 60

print("%d:%02d.%02d" % (hours, minutes, seconds))


print(f'x {x_geo_eq}, y {y_geo_eq}, z {z_geo_eq}')
print(f'RA in time {hours, minutes, seconds}, Decl {Decl}, RA in degress {RA}')

Stjarnhimlen for Mars 19/4/1990: x_mars_helio_ecl 0.9286117857249157, y_mars_helio_ecl 1.1451629263671625, z_mars_helio_ecl 0.0011409986836104686
0:33.28
x 1.0971289548470822, y 0.16138386310115704, z 0.07121211719352274
RA in time (0, 33.472003385224504, 28.32020311347037), Decl 3.6742994658883275, RA in degress 8.368000846306126


In [14]:
def do_MARS(jd_start):
    
    num_days = 10000
    positions = np.zeros((10000,3), dtype='float64')
    
    for i in range(0,num_days):
        jd = jd_start+i
        
        
        ## first compute helio for MARS
        N=49.5574
        N_d=2.11081E-5
        I=1.8497
        I_d=-1.78E-8
        w=286.5016
        w_d=2.92961E-5
        a=1.523688
        a_d=0
        e=0.093405
        e_d=2.516E-9
        M=18.6021
        M_d=0.5240207766
        x_mars_helio_ecl,y_mars_helio_ecl,z_mars_helio_ecl = do_stjarnhimlen(jd,a,e,w,M,a_d,e_d,w_d,M_d,N,N_d,I,I_d)
        
        ## then compute helio for SUN i.e. EARTH
        w = 282.9494
        w_d=4.70935E-5
        a=1
        a_d=0
        e=0.016709 
        e_d=-1.15E-9
        M=356.0470
        M_d=0.9856002585
        N=0
        N_d=0
        I=0
        I_d=0
        x_sun_helio_ecl,y_sun_helio_ecl,z_sun_helio_ecl = do_stjarnhimlen(jd,a,e,w,M,a_d,e_d,w_d,M_d,N,N_d,I,I_d)

        ## from Earth it is sum of x_sun_helio_ecl and x_mars_helio_ecl
        x_geo_ecl = x_mars_helio_ecl + x_sun_helio_ecl
        y_geo_ecl = y_mars_helio_ecl + y_sun_helio_ecl
        z_geo_ecl = z_mars_helio_ecl + z_sun_helio_ecl
        
        positions[i][0] = x_geo_ecl
        positions[i][1] = y_geo_ecl
        positions[i][2] = z_geo_ecl
        ##print(f'i {i}, positions {positions[i]}')
        
    return positions

    

In [15]:
## start at 3000bc
jd_start = 623846.480405
positions = do_MARS(jd_start)
np.savetxt("p.csv", positions, delimiter=",")


In [16]:
colors = np.zeros((10000))
for i in range(1,10000,1):
    colors[i] = i%400

%matplotlib widget

In [17]:
from mpl_toolkits import mplot3d
import itertools

fig = plt.figure(figsize=(20,20))
ax = plt.axes(projection='3d')
ax.scatter(positions[:,0], positions[:,1], positions[:,2], s=5, c=colors)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
def do_rise_set(decl, RA_degrees, latitude, longitude, M_sun, w_sun):
    ## latitude = phi
    ## Decl is angle from equator to point x,y,z
    ## HA is distance between meridian of point and celestial meridian
    ## RA is distance between meridian of point and vernal equinox
    ## Local sidereal time equals hour angle of vernal equinox. LST = RA
    ## LST + longitude west = GST
    ## LHA(star) + longitude west = GHA(star)
    
    ## sin(α) = sin(δ)sin(φ) + cos(δ) cos(φ) cos(H)
    ## If a = 0° (the object is on horizon, either rising or setting), 
    ## then this equation becomes: 
    ## cos(H) = - tan(φ) tan(δ)
    ## This gives the semi-diurnal arc H: 
    ## the time between the object crossing the horizon, and crossing the meridian. 
    cosH = -1*math.tan(decl)*math.tan(latitude*math.pi/180)
    H = math.acos(cosH)
    
    ## H is also diurnal arc so sunrise is noon-H and sunset is noon+H
    
    GST0 = M_sun+w_sun+180  # L+180
    LST = RA_degrees
    #LST = GMST0 + UT*15.0 + longitude
    UT = (LST - GST0 - longitude)/15
    
    ## H + UT = sunset, UT - H = sunrise
    return H, UT

In [52]:
## computer rise/set for MARS for 1-1-2004 from Bangalore lat=12° 58' 18.98", long = 77° 35' 37.28"
Decl = 3.6742994658883275
RA_degrees = 8.368000846306126
latitude = 12.0 + 58/60 + 18.98/3600
longitude = 77 + 35/60 + 37.78/3600
M_sun = 356.9535109211031
w_sun = 283.01824873475516

H, UT = do_rise_set(Decl, RA_degrees, latitude, longitude, M_sun, w_sun)

print(f'H {H}, UT {UT}')

H 1.707023572917555, UT -59.27983910582199


In [ ]:
## verify Mercury
N=48.3313
N_d=3.24587E-5
I=7.0047
I_d=5.00E-8
w=29.1241
w_d=1.01444E-5
a=0.387098
a_d=0
e=0.205635
e_d=5.59E-10
M=168.6562
M_d=4.0923344368
x_mer_helio_ecl,y_mer_helio_ecl,z_mer_helio_ecl = do_stjarnhimlen(jd,a,e,w,M,a_d,e_d,w_d,M_d,N,N_d,I,I_d)
#print(f'Stjarnhimlen for Mercury 19/4/1990: x_mer_helio_ecl {x_mer_helio_ecl}, y_mer_helio_ecl {y_mer_helio_ecl}, z_mer_helio_ecl {z_mer_helio_ecl}')


## from Earth it is sum of x_sun_helio_ecl and x_mer_helio_ecl
x_geo_ecl = x_mer_helio_ecl + x_sun_helio_ecl
y_geo_ecl = y_mer_helio_ecl + y_sun_helio_ecl
z_geo_ecl = z_mer_helio_ecl + z_sun_helio_ecl

## now rotate by obliquity of Eclipise ie. 23 degree tilt to get equatorial
oblecl = 23.4393
x_geo_eq = x_geo_ecl
y_geo_eq = y_geo_ecl * cos(oblecl) - z_geo_ecl * sin(oblecl)
z_geo_eq = y_geo_ecl * sin(oblecl) + z_geo_ecl * cos(oblecl)

RA = arctan_stleimlen(x_geo_eq,y_geo_eq)
Decl = arctan_stleimlen(math.sqrt(x_geo_eq*x_geo_eq + y_geo_eq*y_geo_eq),z_geo_eq)
## print(f'Stjarnhimlen for Mercury 19/4/1990: x_geo_eq {x_geo_eq}, y_geo_eq {y_geo_eq}, z_geo_eq {z_geo_eq}, RA {RA}, decl {Decl}')
